# Clustering de Filmes baseado no resumo dos filmes (synopsis)

# Descrição do trabalho

Neste _notebook_ vamos fazer _clustering_ de filmes baseado na descrição textual (_synopsis_) retirada da base de dados de filmes [IMDb](http://www.imdb.com/).

A similaridade entre filmes será calculada a partir da medida TF-IDF (_Term Frequency - Inverse Document Frequency_) do conteúdo dos resumos dos filmes. Atenção que há vários factores que contribuem para a correcção dos resultados, pelo que o que se pretende neste notebook é exemplificar a metodologia :-)

Resumo das operações realizadas:

1. [Scraping dos 100 melhores filmes](#1.&nbsp;Scraping&nbsp;dos&nbsp;100&nbsp;melhores&nbsp;filmes)
<br>Obter informação dos 100 melhores filmes de todos os tempos (de acordo com este [utilizador](http://www.imdb.com/list/ls055592025/))<br><br>
2. [Tokenizing e stemming](#2.&nbsp;Tokenizing&nbsp;e&nbsp;stemming)
<br> Criar funções que têm como objectivo transformar um documento num conjunto de palavras (um pouco à semelhança do _shingling_). <br><br>
3. [Transformar os dados (tokens) para um espaço vectorial](#3.&nbsp;Transformar&nbsp;os&nbsp;dados&nbsp;para&nbsp;um&nbsp;espaço&nbsp;vectorial) usando a [tf-idf](http://en.wikipedia.org/wiki/Tf%E2%80%93idf)
<br> Após ter um documento representado por um conjunto de palavras, vamos transformar esta representação num vector numérico usando a medida TF-IDF <br><br>
4. [Calcular a _cosine distance_ dos documentos](#4.&nbsp;Calcular&nbsp;a&nbsp;cosine&nbsp;distance) 
<br>Calcular a distância entre todos os documentos usado a TF-IDF e a distância do coseno.<br><br>
5. [Clustering dos documentos](#5.&nbsp;K-means&nbsp;clustering) 
<br>Agrupar os documentos em conjuntos (clusters) usando o algoritmo [k-means](http://en.wikipedia.org/wiki/K-means_clustering)<br><br>
6. [Redução de dimensionalidade](#6.&nbsp;Redução&nbsp;de&nbsp;dimensionalidade) 
<br> Reduzir a dimensionalidade dos dados usando o algoritmo PCA para posterior visualização.
<br><br>
7. [Gráfico dos clusters](#7.&nbsp;Gráfico&nbsp;dos&nbsp;Clusters) 
<br> Visualizar os dados dos clusters usando o [matplotlib](http://matplotlib.org/)
<br><br>
8. [Clustering hierárquico dos filmes](#8.&nbsp;Clustering&nbsp;hierárquico&nbsp;dos&nbsp;filmes) 
<br>Aplicar o algoritmo de _clustering_ hierárquico [Ward clustering](http://en.wikipedia.org/wiki/Ward%27s_method) aos dados.


# Importar as bibliotecas necessárias

In [ ]:
%pylab inline
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import urllib3 
import re
import codecs
import os, shutil
from sklearn import feature_extraction
from sklearn.metrics.pairwise import cosine_similarity

# 1.&nbsp;Scraping&nbsp;dos&nbsp;100&nbsp;melhores&nbsp;filmes

## Download da lista de filmes

In [ ]:
http = urllib3.PoolManager()
r = http.request('GET', "http://www.imdb.com/list/ls055592025/")

soup = BeautifulSoup(r.data, "html.parser")

links = []
titles = []

for i, div in enumerate(soup.find_all('div', {'class': 'info'})):
    for b in div.find_all('b'):
        for a in b.find_all('a'):
            titles.append(a.text)
            links.append(a['href'])
            print(i+1, a.text)
            
print("\nTotal: " + str(len(links)) + ' links e ' + str(len(titles)) + ' títulos de filmes.')    

## Guardar os dados na directoria synopses

Criar a directoria `synopses`. Se a directoria já existe, apagar tudo e criar de novo.

In [ ]:
if os.path.exists('synopses'):
    shutil.rmtree('synopses')
os.mkdir('synopses')

## Fazer o download das synopses

Fazer o download das _synopses_ dos filmes e guardar em ficheiros dentro da directoria `synopses`.

In [ ]:
for i in range(len(titles)):
    
    r2 = http.request('GET', "http://www.imdb.com"+links[i])
    soup2 = BeautifulSoup(r2.data, "html.parser")
    
    divitem = soup2.find('div', {'itemprop': 'genre'})
    genres = []
    for a in divitem.find_all('a'):
        genres.append(a.text.strip())
    
    
    r3 = http.request('GET', "http://www.imdb.com"+links[i]+'plotsummary#synopsis')
    soup3 = BeautifulSoup(r3.data, "html.parser")
    synopsis = soup3.find_all('ul', {'id': 'plot-synopsis-content'})[0].text
    
    
    print('Filme %d: ' %i, titles[i])
    print('Géneros: ', genres)
    print(synopsis[:300].strip()+'[...]'+'\n')
    if not "It looks like we don't have a Synopsis for this title yet" in synopsis[:80].strip():
        with open('synopses/movie_%02d.txt' % i, 'w') as f:
            f.write(synopsis)
    else:
        with open('synopses/movie_%02d.txt' % i, 'w') as f:
            f.write('n.a.')

    with open('synopses/titles.txt', 'a') as f:
        f.write(titles[i] + '\n')
        
    with open('synopses/genres.txt', 'a') as f:
        f.write(str(genres) + '\n')

# 2.&nbsp;Tokenizing&nbsp;e&nbsp;stemming

## Importar os dados

Vamos importar os títulos, os links das páginas, os resumos e os géneros dos filmes.

In [ ]:
filmes = open('synopses/titles.txt').read().strip().split('\n')
genres = open('synopses/genres.txt').read().strip().split('\n')
ranks = [i for i in range(0,len(filmes))]

synopses = []
erased = 0
for i in range(len(filmes)):
    texto = open('synopses/movie_%02d.txt' % i).read().strip('\n')
    if not texto == 'n.a.':
        synopses.append(texto)
    else:
        del ranks[i-erased]
        del genres[i-erased]
        del filmes[i-erased]
        erased +=1

print(str(len(titles)) + ' filmes')


In [ ]:
id_filme = 10
print('Título do filme:', filmes[id_filme])
print('Género:', genres[id_filme])
print('Resumo:', synopses[id_filme])

## Tokenizing

Para obter uma representação dos resumos dos filmes, vamos considerar a frequência das próprias palavras. Para isso é preciso:

- ignorar as _stop words_
- realizar a operação _tokeninzing_, isto é, separar o texto em palavras
- realizar a operação _stemming_, que consiste em representar qualquer palavra pela sua versão _stemmed_, isto, a base que esteve na origem da palavra. Por exemplo, 'runs', 'running' vão ser todas transformadas em 'run'. A ideia é que todas "transmitem" a mesma ideia: correr!

Para realizar as operações atrás descritas, vamos usar a biblioteca [NLTK's](http://www.nltk.org/).
Primeiro vamos importar a lista de [_stop words_](http://en.wikipedia.org/wiki/Stop_words). 

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

De seguida vamos importar o [Snowball Stemmer](http://snowball.tartarus.org/) que faz parte do NLTK.
ou o [Porter Stemmer](http://snowball.tartarus.org/algorithms/porter/stemmer.html))

In [ ]:
# from nltk.stem.porter import PorterStemmer
# stemmer = PorterStemmer()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

Vamos definir duas funções:

- *tokenize_and_stem*: _tokenizes_ (separa o texto em palavras - _tokens_) e realiza a operação _stemming_ a cada palavra (token)
- *tokenize_only*: separa apenas o texto em _tokens_.

Ambas as funções serão usadas mais tarde, uma para criar um dicionário de _stems_ e outra para obter a palavra original. (isto será importante na parte de análise).


In [ ]:
def tokenize_e_stem(texto):
    # A tarefa de obter tokens vai ser realizada em duas fases: primeiro frases e depois palavras.
    tokens = [palavra for frase in nltk.sent_tokenize(texto) for palavra in nltk.word_tokenize(frase)]

    # filtrar tokens que não tenham letras (exemplo: valores numéricos, só pontuação, etc...)
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

# 3.&nbsp;Transformar&nbsp;os&nbsp;dados&nbsp;para&nbsp;um&nbsp;espaço&nbsp;vectorial

## Aplicar as funções aos resumos dos filmes (synopses)

In [ ]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for doc_i in synopses:
    allwords_stemmed = tokenize_e_stem(doc_i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize(doc_i)
    totalvocab_tokenized.extend(allwords_tokenized)

## Relação entre palavras e versão stemmed

Usando estas duas listas, vamos criar um `DataFrame` do `python pandas` com o vocabulário _stemmed_ como índice e os tokens como palavras. A ideia é ter uma forma eficiente de obter a versão completa da palavra a partir da sua versão _stemmed_. O único senão é que não será possível recuperar a palavra original, porque não temos uma relação de um para um. 

In [ ]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

In [ ]:
vocab_frame.head(20)

## TF-IDF and document similarity

<img src='http://www.jiem.org/index.php/jiem/article/viewFile/293/252/2402' align='right' style="margin-left:10px">


Para comparar os diferentes filmes, vamos usar a TF-IDF (_term frequency - inverse document frequency_). A ideia é que uma palavra que apareça muitas vezes num documento é importante. Por outro lado, essa palavra é tão mais importante se aparecer poucas vezes nos documentos todos.

Para obter uma matriz Tf-idf primeiro é necessário contar as occurrências por documento. Depois transforma-se numa matriz _document-term matrix_ (dtm) que está representada à direita na figura. Esta matriz também é conhecida como a _term frequency matrix_.

Algumas notas da implementação:

- `max_df`: valor máximo para a frequência de uma _feature_ dentro de um documento para ser usada na matriz tf-idf. A ideia é que um termo existe em mais de 80% dos documentos, provavelmente não tem grande significado (no contexto dos filmes).
- `min_idf`: valor mínimo com que um _feature_ tem de aparecer na globalidade dos documentos.
- `ngram_range`: qual o tipo de [n-grams](http://en.wikipedia.org/wiki/N-gram) que vamos considerar: unigramas (palavras simples), bigramas (duas palavras de seguida) e trigramas (três palavras de seguida)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8,
                                   max_features=200000,
                                   min_df=0.2, 
                                   stop_words='english',
                                   use_idf=True,
                                   tokenizer=tokenize_e_stem,
                                   ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(synopses)

print('A matris TD-IDF tem dimensões:', tfidf_matrix.shape)

## Ver a lista dos primeiros "tokens"

In [ ]:
terms = tfidf_vectorizer.get_feature_names()
print('Existem ', len(terms), 'termos usados.')
print(terms[:50])

# 4.&nbsp;Calcular&nbsp;a&nbsp;cosine&nbsp;distance

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity <-- o import já foi feito no início do documento
dist = 1 - cosine_similarity(tfidf_matrix)

In [ ]:
print('A matriz de distâncias tem dimensão:', dist.shape)

# 5.&nbsp;K-means&nbsp;clustering

## Agrupar os dados em clusters

Para melhor compreender a estrutura dos dados, nomeadamente dos resumos dos filmes, vamos usar o algoritmo [k-means](http://en.wikipedia.org/wiki/K-means_clustering) usando 5 clusters. 

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters);

%time km.fit(tfidf_matrix)

# passar a classificação para uma lista
clusters = km.labels_.tolist()

## Análise dos resultados

Vamos criar um `DataFrame` com os títulos dos filmes, o rank do filme na lista, o cluster e o género de filme.

In [ ]:
films = { 'titulo': filmes, 'rank': ranks, 'synopsis': synopses, 'cluster': clusters, 'Género': genres }
frame = pd.DataFrame(films, index = [clusters] , columns = ['rank', 'titulo', 'cluster', 'Género'])

In [ ]:
#frame
frame.head()

## Algumas estatísticas
### Número de filmes por cluster

In [ ]:
frame['cluster'].value_counts()

### Valor médio do rank por cluster

In [ ]:
grouped = frame['rank'].groupby(frame['cluster'])
grouped.mean()

### Top "palavras" por cluster

Vamos mostrar, para cada cluster, quais as palavras que definem o cluster e que filmes se encontram em cada cluster

In [ ]:
cluster_words =[[] for i in range(num_clusters)]
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    #print("-" * 80)
    print("Palavras do cluster %d:" % i, end='')
    for ind in order_centroids[i, :6]:
        print(' %s,' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0], end='')
        if len(cluster_words[i])<3:
            cluster_words[i].append(vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0])
    print("\nFilmes do cluster %d:" % i)
    for title in frame.ix[i]['titulo'].values.tolist():
        print('   - %s' % title)
    print("\n")

# 6.&nbsp;Redução&nbsp;de&nbsp;dimensionalidade

Para representar os filmes, vamos reduzir a dimensionalidade do problema, transformado a matriz de distâncias num array bi-dimensional usando o algoritmo PCA.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
#pos = pca.fit(X).transform(X)
pos = pca.fit_transform(dist)
xs, ys = pos[:, 0], pos[:, 1]

# 7.&nbsp;Gráfico&nbsp;dos&nbsp;Clusters

## Definir o nome e as cores do cluster

In [ ]:
# Definir a cor dos clusters
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e'}

# Definir o nome dos cluster (usando um dicionário)
cluster_names = { i: cluster_words[i] for i in range(num_clusters)}

## Organizar os dados num DataFrame

In [ ]:
# cirar um dataframse que seja o resultado do PCA, o número dos clusters e o título dos filmes
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=filmes)) 

# agrupar por cluster
groups = df.groupby('label')

In [ ]:
groups.head()

## Mostrar o gráfico

In [ ]:
# set up plot
fig, ax = plt.subplots(figsize=(17, 9)) # set size
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=cluster_names[name], color=cluster_colors[name], mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',         # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='off',        # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelleft='off')
    
ax.legend(numpoints=1)  #show legend with only 1 point

#add label in x,y position with the label as the film title
for i in range(len(df)):
    ax.text(df.ix[i]['x'], df.ix[i]['y'], df.ix[i]['title'], size=8)  

    
    
plt.show() #show the plot

#A linha seguinte permite gravar o gráfico como uma imagem
#plt.savefig('cluster_filmes.png', dpi=200)

# 8.&nbsp;Clustering&nbsp;hierárquico&nbsp;dos&nbsp;filmes

Vamos agora usar _hierarchical clustering_ para representar a mesma informação. O algoritmo usado neste exemplo é o [Ward clustering algorithm](http://en.wikipedia.org/wiki/Ward%27s_method). 

In [ ]:
from scipy.cluster.hierarchy import ward, dendrogram

linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

fig, ax = plt.subplots(figsize=(15, 30)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=filmes, leaf_font_size=15);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout

#para gravar a figura:
#plt.savefig('cluster_filmes_hiearquico.png', dpi=200) 